# let's try Joensuu !

input search parameters:
- area of interest: [29.708405, 62.585736, 29.816895, 62.629308]
- time range: 
    - start_date: 2023-07-01
    - end_date: 2023-08-14
- satellite collection: sentinel-2

In [ ]:
# STAC access
import pystac_client
import planetary_computer

# library for handling GIS rasters
import rasterio as rio
import rasterio.mask

from matplotlib import pyplot as plt
import numpy as np
import rich

In [ ]:
# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

In [ ]:
aoi = [29.708405, 62.585736, 29.816895, 62.629308]
#time_range
start_date='2023-07-01'
end_date='2023-08-14'
sat_coll=['sentinel-2-l2a']

In [ ]:
search = catalog.search(
    collections=sat_coll,
    datetime=f'{start_date}/{end_date}',
    bbox=aoi
)
items = search.item_collection()
print(f"{len(items)} items found with the `bbox` parameter")

In [ ]:
for img in items:
    print(img.id, img.datetime)

In [ ]:
item = catalog.get_collection('sentinel-2-l2a').get_item("S2A_MSIL2A_20230812T094041_R036_T36VUQ_20230812T161209")
item

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    profile = ds.profile
rich.print(profile)


In [ ]:
converted_aoi_bounds = rio.warp.transform_bounds("epsg:4326", profile['crs'], *aoi)
converted_aoi_bounds

In [ ]:
aoi_window = rio.windows.from_bounds(*converted_aoi_bounds, transform = profile['transform'])
aoi_window

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    band_blue = ds.read(window=aoi_window)
with rio.open(item.assets['B03'].href) as ds:
    band_green = ds.read(window=aoi_window)
with rio.open(item.assets['B04'].href) as ds:
    band_red = ds.read(window=aoi_window)
with rio.open(item.assets['B08'].href) as ds:
    band_nir = ds.read(window=aoi_window)

In [ ]:
def normalize(array, vmax=False):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    if vmax:
        array[array>vmax] = vmax
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

band_blue_n = normalize(band_blue, 3000)
band_green_n = normalize(band_green, 3000)
band_red_n = normalize(band_red, 3000)
band_nir_n = normalize(band_nir, 3000)

In [ ]:
rgb = np.stack((band_red_n, band_green_n, band_blue_n))
rgb_natcol = np.moveaxis(rgb.squeeze(), 0, -1)
rgb = np.stack((band_nir_n, band_red_n, band_green_n))
rgb_nircol = np.moveaxis(rgb.squeeze(), 0, -1)

In [ ]:
f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow(rgb_natcol)
plt.title('Joensuu: natural color')
f.add_subplot(1,2, 2)
plt.imshow(rgb_nircol)
plt.title('Joensuu: nir false color')
plt.show(block=True)
